Importovanie knižníc, funkcie na úpravu dátumu a nastavenie adresy pre ElasticSearch

In [1]:
from bs4 import BeautifulSoup
import re
import os
import json
import calendar
import glob
from elasticsearch import Elasticsearch
import datetime
abbr_to_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [2]:
with open('analyzer.txt', 'r') as myfile:
    analyzer=myfile.read()
with open('mapping.txt', 'r') as myfile:
    mapping=myfile.read()
es.indices.create(index='goodreads-test', ignore=400, body=analyzer)
es.indices.put_mapping(index='goodreads-test',body=mapping)

{'acknowledged': True}

Funkcia na extrahovanie informácií z HTML stánky spolu s ošetrením, ak sa daný údaj na stránke nenachádza.

In [3]:
def parse_html(html):
#     print(html)
#     try

    try:
        ident = re.search('\/book\/show\/([0-9]*)',html).group(1)
#         print(ident)
    except: 
        ident = None
    try:
        description = re.search('id=\"description\"(.*\n){3}.*\"\>(.*)\<\/',html).group(1)
#         print(description)
    except: 
        try:
            description = re.search('id=\"description\"(.*\n){1}.*\"\>(.*)\<\/',html).group(2)
        except:
            description = None
    try:
#         description = description.replace('<[^>]*>', ' ')
        description = re.sub('<[^>]*>', " ", description)
    except: 
        description = None
    try:
        name = re.search('\"bookTitle.*\n\s*(.*)',html).group(1)
#         name = re.search('(?<=(Original Title<\/div>\n)).*(?=\ \<)',html)
        print(name)
    except: 
        name = None
#         print("sas")
    try:
        isbn = re.search('(?<=(ISBN<\/div>\n)).*\n\s*(.*)',html).group(2)
    except: 
        isbn = None
    try:
        isbn13 = re.search('(?<=(ISBN<\/div>\n)).*\n.*\n.*((?<=\>).*)(?=\<span)[^0-9]*([0-9]+)',html).group(3)
    except: 
        isbn13 = None
    try:
        series = re.search('(?<=(Series<\/div>\n))((\s*\<d|\s*\<a).*\n){0,10}\s*<\/div>',html).group()
    except: 
        series = None
    try:
        series = re.findall('\>([a-zA-Z0-9#\ -]*)\<\/a',series)
    except: 
        series = None
    try:
        characters = re.findall('\/characters.{1,40}"\>([a-zA-z0-9\ ]*)\<',html)
    except: 
        characters = None
#     obj= make_BookObj(ident, name, isbn, isbn13, series, characters)
#     Object.assign(obj, characters)

#     return obj
#     print(create_json(ident, name, isbn, isbn13, series, description, characters))
    return create_json(ident, name, isbn, isbn13, series, description, characters)

In [4]:
class BookObj(object):
    ident: 0
    name: None
    isbn: None
    isbn13: None
    series: None
#          description: description
    characters: None

    def __init__(self, ident, name, isbn, isbn13, series, characters):
        self.ident: ident
        self.name: name
        self.isbn: isbn
        self.isbn13: isbn13
        self.series: series
#          self.description: description
        self.characters: characters

def make_BookObj(ident, name, isbn, isbn13, series, characters):
    book = BookObj(ident, name, isbn, isbn13, series, characters)
    return book

Funkcia zo vstupných údajov vytvorí JSON formát, a vlozi ho do elastic search databazy.

In [5]:
def create_json(ident, name, isbn, isbn13, series, description, characters):
    x = {
      "id": ident,
      "name": name,
      "isbn": isbn,
      "isbn13": isbn13,
      "series": series,
      "description": description,
      "characters": characters
    }
    y = json.dumps(x)
    global es
    page_id=0
    page_id += 1
    return y
#     print(json.dumps(json.loads(y), indent=2))

Spracovanie všetkých HMTL súborov uložených v adresári.

In [6]:
folders = os.listdir("D:/School/VINF/Data")
arr = []

for folder in folders:
    book = {}
    arr.clear()
    book['book'] = folder
    files = glob.glob("D:/School/VINF/Data/"+folder+"/*.html")
    for file in files:
        html = open(file,encoding="utf8")    
        soup = html.read()
#         print(soup)
        obj = (json.loads(parse_html(soup)))
        ident= re.search('([0-9]*)\.',file).group(1)
        obj['id'] = ident
        arr.append(obj)
        book['editions']=arr
#         print(arr)
    jsonObj=json.dumps(book, indent=4, )
    es.index(index='goodreads-test', id=ident, body=jsonObj)
#     print(jsonObj)
#     print("  ")

Cave Life in France: Eat, Drink, Sleep...
Jeanne Demessieux (1921--1968): A critical examination of her life (France) -- Dissertation
Euripides I: Alcestis / The Medea / The Heracleidae / Hippolytus
Medea and Other Plays
Perspective Made Easy
Easy Spanish Phrase Book: Over 770 Basic Phrases for Everyday Use
The Easy Way to Stop Drinking
Diabetes Meal Planning Made Easy
Easy French Reader
Lao Tzu: Tao Te Ching: A Book about the Way and the Power of the Way
The Daodejing of Laozi
The Easy Piano Christmas Collection: 59 Holiday Classics
Fluids and Electrolytes Made Incredibly Easy!
Gem Identification Made Easy: A Hands-On Guide to More Confident Buying &amp; Selling
Senior's Guide to Easy Computing
An Thomas Troward Intimate Memoir of the Teacher and the Man
I Remember As Though It Were Yesterday: The Stories Of Greg Charlesworth:  The Athlete, The Coach, The Teacher, The Man
Journalism Toady Teachers Man
Easy Exotic: A Model's Low-Fat Recipes from Around the World
Math Made Easy: Third G

Easy Gourmet Cooking with 5 Ingredients
Road Kill Cooking Gourmet Style
Gourmet Style Road Kill Cooking and Other Fine Recipes
Gourmet Cuisine Island Style
The Cape Cod Fish &amp; Seafood Cookbook: From basic to Gourmet
Rocky Mountain Gourmet Cookbook
Gourmet's Guide to New Orleans: Creole Cookbook
Journal, Volume 6
Prince of Ice
Emma Lazarus
The Long Walk: The True Story of a Trek to Freedom
Old Town in the Green Groves: Laura Ingalls Wilder's Lost Little House Years
A Feast for Crows
Emma's Gift
Love's Hidden Blessings: God Can Touch Your Life When You Least Expect It
With Open Hands
All Things Are Possible: Gift Boxed
Hot Money
This Is the Day
Trial Run
Saxons, Vikings, and Celts: The Genetic Roots of Britain and Ireland
Yeah, I Said It
Cat's Eye
What Did I Do Last Night?: A Drunkard's Tale
Pricing Without Fear
The Beach
La Plage
Der Strand
Manila
The Tesseract
Dumbing Down Our Kids: Why American Children Feel Good About Themselves But Can't Read, Write or Add
Beauty's Punishment
No

Die Langsamkeit
Prodigal Son
Bookclub-in-a-Box Discusses The Poisonwood Bible, the Novel by Barbara Kingsolver
Barbara Kingsolver's The Poisonwood Bible: A Reader's Guide
A Reader's Guide to the Poisonwood Bible
The President's House: A First Daughter Shares the History and Secrets of the World's Most Famous Home
How to Make Love Like a Porn Star: A Cautionary Tale
How to Make Love Like a Porn Star CD: A Cautionary Tale
Cómo hacer el amor igual que una estrella porno
Jenna Jameson J2K
Fires: Essays, Poems, Stories
Call If You Need Me: The Uncollected Fiction and Other Prose
Will You Please Be Quiet, Please?
Il Mondo Della Giada: I Grandi Capolavori Dell'arte Cinese
Guided interpretations and the importance of signs: Text, reader, and author in Carlos Fuentes and Jorge Luis Borges
Dominion: The Power of Man, the Suffering of Animals, and the Call to Mercy
Ivanhoe
Ivanhoe
Ivanhoe
Ivanhoe
Ivanhoe
Ivanhoe
Everlasting Dominion: A Theology of the Old Testament
No Dominion
Dominion
Manly Domi

When Buildings Speak: Architecture as Language in the Habsburg Empire and Its Aftermath, 1867-1933
The Shaping of America: A Geographical Perspective on 500 Years of History: Volume 2: Continental America, 1800-1867
Industrial Reformation of English Fiction: Social Discourse and Narrative Form, 1832-1867
From New Deal Banking Reform to World War II Inflation. Reprinted from the Author's Monetary History of the United States, 1867-1960
Paris: 500 Photos
Paris: 500 Photos
The Basic Bakunin
A Victorian Housebuilder's Guide: Woodward's National Architect of 1869
When the Heart Waits: Spiritual Direction for Life's Sacred Questions
The Gold Ring: Jim Fisk, Jay Gould, and Black Friday, 1869
The Negro Leagues, 1869-1960
Swiss Magazine Loading Rifles 1869 to 1958, 2nd edition, revised
Black Southerners, 1619-1869
Mendeleev on the Periodic Law: Selected Writings, 1869 - 1905
Conceiving a New Republic: The Republican Party and the Southern Question, 1869-1900 (American Political Thought)
Retreat

Un Fuego Oculto: 14 Historias Cortas
Diario Del Rodaje De Al Sur De Granada
La Pintura En El Cine, El Cine En La Pintura
Songs of the Humpback Whale
Bold Love Discussion Guide
Michael Crichton: A Critical Companion
Five Go Adventuring Again
Five Get into a Fix
The Secret of Santa Vittoria
Better Than Sex: Confessions of a Political Junkie
McSweeney's #23
Peter Pan and Other Plays
Four Comedies: Lysistrata / The Frogs / The Birds / Ladies' Day
The Firm of the Future: A Guide for Accountants, Lawyers, and Other Professional Services
Creating Dominance: Winning Strategies for Law Firms
Elizabeth Costello
The First Great Myth of Legal Management Is That It Exists: Tough Issues for Law Firm Managing Partners and Administrators
Jo's Troubled Heart
The Electric Kool-aid Acid Test/The Kandy Kolored Tangerine Flake Streamline Baby/Radical Chic &amp; Mau Mauing the Flak Catchers
Management Information Systems: Managing the Digital Firm
Architect's Essentials of Starting, Assessing and Transition

...

The Facts
Combat Zone
In The Combat Zone: An Oral History of American Women in Vietnam, 1966-1975
If I Die in a Combat Zone, Box Me Up and Ship Me Home
If I Die in a Combat Zone
If I Die In A Combat Zone
If I Die in a Combat Zone box me up and ship me home
From Combat Zone to Love at Home
Tom Clancy's Rainbow Six Vegas (Prima Official Game Guide)
Tom Clancy's Rainbow Six: Lockdown: Prima Official Game Guide (Prima Official Game Guides)
Watermelon
The Black Veil: A Memoir with Digressions
Black Veil: A Memoir with Digressions
Garden State
Tom Clancy's Rainbow Six Lockdown (PC): Prima Official Game Guide (Prima Official Game Guides)
Tom Clancy's Rainbow Six 3 (PS2): Prima's Official Strategy Guide
Tom Clancy's Rainbow Six Bundle for Red Storm: Prima's Official Strategy Guide
Go Down, Moses
Child of War, Woman of Peace
When Heaven and Earth Changed Places: A Vietnamese Woman's Journey from War to Peace
When Heaven and Earth Changed Places: A Vietnamese Woman's Journey from War to Peace
Wr

Not a Sparrow Falls
Sirens and Sea Monsters
Aleutian Sparrow
Dance of the Hours
Horton Hears a Who!
¡Horton escucha a quién!
Li Po and Tu Fu: Poems
A Floating Life
Zodiac
Dharma Bell And Dharani Pillar: Li Po? Buddhist Inscriptions
Ivre De Tao: Li Po, Voyageur, Poète Et Philosophe, En Chine, Au Vii Ie Siècle
Li Po-Yuan
Out of Phaze
Banished Immortal: Visions of Li T'Ai-Po
Le Livre De Li Po: Roman
Roc and a Hard Place
Treasure of Li-Po
The Poetry and Career of Li Po
The Works Of Li Po: The Chinese Poet (1922)
The Poet Li Po
Li Pai: 200 Selected Poems
Ascended Master Dictations 2: Talks with the Masters
Ascended Master Dictations: Letters to a Chela
Wave Hands Like Clouds: Kuang Ping Tai Chi: A Chinese Yoga of Meditation in Motion
Five T'ang Poets
Bright Moon, Perching Birds: Poems
Endless River
Endless Rivers: From darkness to light
The Endless River
Beyond the Endless River: Western American Drawings &amp; Watercolors of the Nineteenth Century
The Selected Poems of Tu Fu
The Selected

Inner Workings: Literary Essays 2000-2005
Significant Others
Sure of You
Writing Out of All the Camps: J.M. Coetzee's Narratives of Displacement
J.M. Coetzee: South Africa and the Politics of Writing
Killing Pablo: The  Hunt for the World's Greatest Outlaw.
Matar a Pablo Escobar/Killing Pablo
Killing Pablo: The Hunt for the World's Greatest Outlaw
Killing Pablo: The Hunt for the World's Greatest Outlaw
The Wife
The Mistress of Husaby
The Axe
Jenny
The Sherlock Holmes Mysteries
Sigrid Undset in America: An Annotated Bibliography and Research Guide
The Unknown Sigrid Undset: Jenny and Other Works
Return to the Future
Gunnar's Daughter
The Snake Pit
The Son Avenger
Sigrid Undset, et liv (Norwegian Edition)
Sigrid Undset: Chronicler of Norway (Berg Women's Series)
Sigrid Undsets "Kristin Lavransdatter": Kristins religøse utvikling
Point Counter Point
Six Scandinavian Novelists: Lie, Jacobsen, Heidenstam, Selma Lagerlof, Hamsum, Sigrid Undset
彼方から 11
Sigrid Undset: Une Biographie
Livssyn Og

Nima: A Sherpa in Connecticut
My Brown Bear Barney at the Party
Mating Cries
Me &amp; Jezebel
Me and Jezebel: Two Person and One Person Versions
Our Voices, Our Visions
Me and Jezebel: When Bette Davis Came for Dinner -- And Stayed ... And Stayed ... And Stayed ... And ...
George Washington and Benedict Arnold: A Tale of Two Patriots
George Washington's Socks
Writings
Fraud: Essays
The Unexpected George Washington
The George Washington Collection: Fine and Decorative Arts at Mount Vernon
New England and the Bavarian Illuminati
George Washington Smith: An Architect's Scrapbook
Dubin's Lives
Saint Augustine
A Year in the Life of William Shakespeare: 1599
Meet George Washington
An Autobiography of George Washington
George Washington and the General's Dog (Step Into Reading)
George Washington's Sacred Fire
George Washington's World
George Washington Carver: His Life &amp; Faith in His Own Words
Maisie Dobbs
Selected Writings of Sir Walter Scott
Wilfred Owen
Happiness, Death, and the Remain

Eaters of the Dead
Turbulences
Airframe
Airframe
The Andromeda Strain
Papillions
Perfume Bottles
Les Papillions 1994
Cajun For Kids
A Million and One Love Strategies
Wild Things
The Long Goodbye
The Long Goodbye
The Long Goodbye
The Long Goodbye
Wild Things They Don't Tell Us
Illuminating Angels  Demons: The Unauthorized Guide to the Facts Behind Dan Brown's Bestselling Novel
The Truth About Angels and Demons
All Things Wild And Wonderful
Predator Calling with Gerry Blair
Wild Things: Untold Tales from the First Century of the Saint Louis Zoo
The Bounty of the Earth Cookbook: How to Cook Fish, Game, and Other Wild Things
Wild Things
Wild Berries and Other Wild Things
500 Self-Portraits
Understanding David Foster Wallace
Find Out about France


IndentationError: expected an indented block (<ipython-input-13-0947957eee0c>, line 7)